In [673]:
import numpy as np
import re

with open("../data/day18.txt", "r") as f:
    input = f.read()

data = input.split("\n")

### Puzzle 1

In [684]:
# Helpers
def parenthetic_contents(fullstr):
    """Generate parenthesized contents in string as pairs (level, contents).
    
    https://stackoverflow.com/questions/4284991/parsing-nested-parentheses-in-python-grab-content-by-level
    """
    stack = []
    for i, c in enumerate(fullstr):
        if c == '[':
            stack.append(i)
        elif c == ']' and stack:
            start = stack.pop()
            yield (len(stack), (start + 1, i), fullstr[start + 1: i])


def find_nested_pair(fullstr):
    """Returns (span left, span right, left number, right number)."""
    tree = parenthetic_contents(fullstr)

    for (level, pos, string) in tree:
        results = re.search("(\d{1,}),(\d{1,})", string)
        if level == 4 and results:
            return (pos[0], pos[1], int(results.group(1)), int(results.group(2)))
    return None


def find_number_from(fullstr, pos, direction='right'):
    if direction == 'right':
        right = fullstr[pos+1:]
        result = re.search('\d{1,}', right)
        if result:
            return tuple(pos+1 + span for span in result.span())
    elif direction == 'left':
        left = fullstr[:pos][::-1]
        result = re.search('\d{1,}', left)
        if result:
            return tuple(pos - span for span in result.span()[::-1])
    return None


def addinstr(fullstr, span, number: int):
    return fullstr[:span[0]] + str(int(fullstr[span[0]:span[1]]) + number) + fullstr[span[1]:]

# Actions
def explode(fullstr):
    if not find_nested_pair(fullstr):
        return fullstr, False
    left_pos, _, left_value, _ = find_nested_pair(fullstr)
    span = find_number_from(fullstr, left_pos, direction="left")
    if span:
        fullstr = addinstr(fullstr, span, left_value)

    _, right_pos, _, right_value = find_nested_pair(fullstr) # Re-do because length might change
    span = find_number_from(fullstr, right_pos, direction="right")
    if span:
        fullstr = addinstr(fullstr, span, right_value)
    
    left_pos, right_pos , _, _ = find_nested_pair(fullstr) # Re-do because length might change
    fullstr = fullstr[:left_pos-1] + '0' + fullstr[right_pos+1:]
    return fullstr, True


def split(fullstr):
    results = re.search('\d{2,}', fullstr)
    if not results:
        return fullstr, False
    span = results.span(0)
    number = int(fullstr[span[0]:span[1]])
    left_number = int(np.floor(number/2))
    right_number = int(number - left_number)
    return fullstr[:span[0]] + f'[{left_number},{right_number}]' + fullstr[span[1]:], True


def add(a, b):
    return f'[{a},{b}]'

# Reduce
def reduce_once(fullstr):
    result, done_it = explode(fullstr)
    if not done_it:
        result, done_it = split(fullstr)
    return result, done_it


def reduce(string):
    done_it = True
    while done_it:
        string, done_it = reduce_once(string)
    return string

# Magnitude
def magnitude_once(string):
    results = re.search("(\d{1,}),(\d{1,})", string)
    return 3 * int(results.group(1)) + 2 * int(results.group(2))


def magnitude(string):
    while True:
        matches = re.findall('\[\d{1,},\d{1,}\]', string)
        if not matches:
            break
        for match in matches:
            string = string.replace(match, str(magnitude_once(match)))
    return int(string)

In [639]:
# Test explode
test_input = [
    '[[[[[9,8],1],2],3],4]',
    '[7,[6,[5,[4,[3,2]]]]]',
    '[[6,[5,[4,[3,2]]]],1]',
    '[[3,[2,[1,[7,3]]]],[6,[5,[4,[3,2]]]]]',
    '[[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]',
    '[[[[0,7],4],[7,[[8,4],9]]],[1,1]]' # later example
]

test_output = [
    '[[[[0,9],2],3],4]',
    '[7,[6,[5,[7,0]]]]',
    '[[6,[5,[7,0]]],3]',
    '[[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]',
    '[[3,[2,[8,0]]],[9,[5,[7,0]]]]',
    '[[[[0,7],4],[15,[0,13]]],[1,1]]'
]

for input, output in zip(test_input, test_output):
    print('-------- Test --------')
    print(f'Input:    {input}')
    print(f'Result:   {explode(input)[0]}')
    print(f'Expected: {output}')


-------- Test --------
Input:    [[[[[9,8],1],2],3],4]
Result:   [[[[0,9],2],3],4]
Expected: [[[[0,9],2],3],4]
-------- Test --------
Input:    [7,[6,[5,[4,[3,2]]]]]
Result:   [7,[6,[5,[7,0]]]]
Expected: [7,[6,[5,[7,0]]]]
-------- Test --------
Input:    [[6,[5,[4,[3,2]]]],1]
Result:   [[6,[5,[7,0]]],3]
Expected: [[6,[5,[7,0]]],3]
-------- Test --------
Input:    [[3,[2,[1,[7,3]]]],[6,[5,[4,[3,2]]]]]
Result:   [[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]
Expected: [[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]
-------- Test --------
Input:    [[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]
Result:   [[3,[2,[8,0]]],[9,[5,[7,0]]]]
Expected: [[3,[2,[8,0]]],[9,[5,[7,0]]]]
-------- Test --------
Input:    [[[[0,7],4],[7,[[8,4],9]]],[1,1]]
Result:   [[[[0,7],4],[15,[0,13]]],[1,1]]
Expected: [[[[0,7],4],[15,[0,13]]],[1,1]]


In [640]:
# Test reduce
a = '[[[[4,3],4],4],[7,[[8,4],9]]]'
b = '[1,1]'
string = reduce(add(a,b))
string

'[[[[0,7],4],[[7,8],[6,0]]],[8,1]]'

In [685]:
# Test multiple sums
test = [
    '[[[0,[4,5]],[0,0]],[[[4,5],[2,6]],[9,5]]]',
    '[7,[[[3,7],[4,3]],[[6,3],[8,8]]]]',
    '[[2,[[0,8],[3,4]]],[[[6,7],1],[7,[1,6]]]]',
    '[[[[2,4],7],[6,[0,5]]],[[[6,8],[2,8]],[[2,1],[4,5]]]]',
    '[7,[5,[[3,8],[1,4]]]]',
    '[[2,[2,2]],[8,[8,1]]]',
    '[2,9]',
    '[1,[[[9,3],9],[[9,0],[0,7]]]]',
    '[[[5,[7,4]],7],1]',
    '[[[[4,2],2],6],[8,7]]',
]

string = test[0]
for pair in test[1:]:
    string = reduce(add(string, pair))

print(f'Result:   {string}')
print("Expected: [[[[8,7],[7,7]],[[8,6],[7,7]]],[[[0,7],[6,6]],[8,7]]]")

Result:   [[[[8,7],[7,7]],[[8,6],[7,7]]],[[[0,7],[6,6]],[8,7]]]
Expected: [[[[8,7],[7,7]],[[8,6],[7,7]]],[[[0,7],[6,6]],[8,7]]]


In [686]:
# Test magnitude
magnitude('[[[[8,7],[7,7]],[[8,6],[7,7]]],[[[0,7],[6,6]],[8,7]]]')

3488

In [687]:
# Final test
test = [
    '[[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]]',
    '[[[5,[2,8]],4],[5,[[9,9],0]]]',
    '[6,[[[6,2],[5,6]],[[7,6],[4,7]]]]',
    '[[[6,[0,7]],[0,9]],[4,[9,[9,0]]]]',
    '[[[7,[6,4]],[3,[1,3]]],[[[5,5],1],9]]',
    '[[6,[[7,3],[3,2]]],[[[3,8],[5,7]],4]]',
    '[[[[5,4],[7,7]],8],[[8,3],8]]',
    '[[9,3],[[9,9],[6,[4,9]]]]',
    '[[2,[[7,7],7]],[[5,8],[[9,3],[0,2]]]]',
    '[[[[5,2],5],[8,[3,7]]],[[5,[7,5]],[4,4]]]',
]

string = test[0]
for pair in test[1:]:
    string = reduce(add(string, pair))

print(string)
magnitude(string)

[[[[6,6],[7,6]],[[7,7],[7,0]]],[[[7,7],[7,7]],[[7,8],[9,9]]]]


4140

In [688]:
# Puzzle 1
string = data[0]
for pair in data[1:]:
    string = reduce(add(string, pair))

print(string)
magnitude(string)

[[[[7,7],[8,8]],[[0,8],[9,8]]],[[9,3],[8,[2,6]]]]


2907

### Puzzle 2

In [689]:
test = [
    '[[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]]',
    '[[[5,[2,8]],4],[5,[[9,9],0]]]',
    '[6,[[[6,2],[5,6]],[[7,6],[4,7]]]]',
    '[[[6,[0,7]],[0,9]],[4,[9,[9,0]]]]',
    '[[[7,[6,4]],[3,[1,3]]],[[[5,5],1],9]]',
    '[[6,[[7,3],[3,2]]],[[[3,8],[5,7]],4]]',
    '[[[[5,4],[7,7]],8],[[8,3],8]]',
    '[[9,3],[[9,9],[6,[4,9]]]]',
    '[[2,[[7,7],7]],[[5,8],[[9,3],[0,2]]]]',
    '[[[[5,2],5],[8,[3,7]]],[[5,[7,5]],[4,4]]]',
]

print(test[8])
print(test[0])

[[2,[[7,7],7]],[[5,8],[[9,3],[0,2]]]]
[[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]]


In [690]:
def magnitude_two_numbers(a, b):
    return magnitude(reduce(add(a,b)))

magnitude_two_numbers(test[8], test[0])

3993

In [691]:
from itertools import permutations

pairs = permutations(test, r=2)
magnitudes = [magnitude_two_numbers(pair[0], pair[1]) for pair in pairs]
max(magnitudes)

3993

In [692]:
pairs = permutations(data, r=2)
magnitudes = [magnitude_two_numbers(pair[0], pair[1]) for pair in pairs]
max(magnitudes)

4690

### Appendix

In [ ]:
# Puzzle 1
import json
js = json.loads('[[3,[2,[1,[7,3]]]],[6,[5,[4,[3,2]]]]]')
js

[[3, [2, [1, [7, 3]]]], [6, [5, [4, [3, 2]]]]]

In [ ]:
from treelib import Node, Tree
tree = Tree()
tree.create_node("Harry", "harry")  # root node
tree.create_node("Jane", "jane", parent="harry")
tree.create_node("Bill", "bill", parent="harry")
tree.create_node("Diane", "diane", parent="jane")
tree.create_node("Mary", "mary", parent="diane")
tree.create_node("Mark", "mark", parent="jane")
tree.show()

Harry
├── Bill
└── Jane
    ├── Diane
    │   └── Mary
    └── Mark

